In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [ ]:
kernel = sk.Kernel()

deployment = 'deploymentname'
api_key = 'apikey'
endpoint = 'openaiURL'

kernel.add_chat_service("chat", AzureChatCompletion(deployment, endpoint, api_key))

base_skills_directory = '/lakehouse/default/Files/Skills'

skills = {
**kernel.import_semantic_skill_from_directory(base_skills_directory , "dataskill"),
}

csv_skill = skills["category"]

In [ ]:
import pandas as pd
df = pd.read_csv("/lakehouse/default/Files/csv/ProductList.csv")

In [ ]:
display(df)

In [ ]:
IS_CUSTOM_DATA = False  # if TRUE, dataset has to be uploaded manually

DATA_FOLDER = "Files/amazon-products"  # folder with data files
DATA_FILE = "amazon_products.csv"  # data file name

In [ ]:
if not IS_CUSTOM_DATA:

    import os, requests
    # Download demo data files into lakehouse if not exist
    remote_url = "https://synapseaisolutionsa.blob.core.windows.net/public/AmazonProducts"
    fname = "amazon_products.csv"
    download_path = f"/lakehouse/default/{DATA_FOLDER}/raw"

    if not os.path.exists("/lakehouse/default"):
        raise FileNotFoundError("Default lakehouse not found, please add a lakehouse and restart the session.")
    os.makedirs(download_path, exist_ok=True)
    if not os.path.exists(f"{download_path}/{fname}"):
        r = requests.get(f"{remote_url}/{fname}", timeout=30)
        with open(f"{download_path}/{fname}", "wb") as f:
            f.write(r.content)
    print("Downloaded demo data files into lakehouse.")

In [ ]:
import pandas as pd
df = pd.read_csv(f"/lakehouse/default/{DATA_FOLDER}/raw/{DATA_FILE}")

In [ ]:
# Create a sample DataFrame
df = df.sample(frac=0.2, random_state=0).head(7)

In [ ]:
display(df)

In [ ]:
json_str = df.to_json(orient='records')

In [ ]:
panda_context_variables = sk.ContextVariables(variables={
"question": f"""
    Could you determine the product category for each product?
    Please use all information available in the dataframe to determine the product category as if this is going to be sold on Amazon.
    The `Context` data below is the data from the dataframe
    Provide multiple categories separated by a comma if multiple categories are approprate.
    If you are unsure or a category cannot be determined, say "Unknown".
    Write the category as a single word or short phrase.
    Examples:
    DC Cover Girls: Black Canary by Joëlle Jones Statue: Toys,
    Pacific Play Tent Agility Dog Training Chute: Pet Supplies.
    Context:
    {json_str}"""
})
panda_result = await csv_skill.invoke_async(variables=panda_context_variables )
pandasql = panda_result .result.replace("\\n","").replace("\\n\\n", "").replace("print(","display(")
exec(pandasql)

In [ ]:
print(pandasql)

In [ ]:
display(df)